In [ ]:
%pip install fuzzywuzzy torch pickle datasets transformers

In [ ]:
import torch
import pickle
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from fuzzywuzzy import fuzz  # Install with: pip install fuzzywuzzy

In [ ]:
# Constants
MODEL_NAME = "google/gemma-2-2b"
DATASET_NAME = "pminervini/NQ-Swap"
SPLIT = "dev"
SIMILARITY_THRESHOLD = 90  # Adjust as needed

In [ ]:
# Load dataset
print("Loading dataset...")
dataset = load_dataset(DATASET_NAME, split=SPLIT)

# Load model and tokenizer
print("Loading model and tokenizer...")
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

In [ ]:
def generate_answer(question):
    """Generates an answer using the model."""
    inputs = tokenizer(question, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Check memorization
memorised_indices = []
print("Processing dataset...")

for idx, item in enumerate(dataset):
    question = item["question"]
    correct_answer = item["answer"]

    # Generate the model's response
    model_answer = generate_answer(question)

    # Check if the model's response is correct
    if fuzz.ratio(model_answer.lower(), correct_answer.lower()) >= SIMILARITY_THRESHOLD:
        memorised_indices.append(idx)

    # Print progress every 100 samples
    if idx % 100 == 0:
        print(f"Processed {idx}/{len(dataset)} samples. Memorized so far: {len(memorised_indices)}")

In [ ]:
# Save memorised indices to a .pkl file
output_file = "nqswap-gemma-2-2b-memorised_set"
with open(output_file, "wb") as f:
    pickle.dump(memorised_indices, f)

print(f"Saved {len(memorised_indices)} memorised samples to {output_file}")